<a href="https://colab.research.google.com/github/manikcb/sem_project/blob/main/cyberbully30066.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q -U "tensorflow-text==2.8.*"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.1/498.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.3/462.3 KB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 98.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 91.9 MB/s eta 0:00:00


In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd


In [4]:
df = pd.read_csv("/content/drive/MyDrive/dataset/Untitled folder/mydata3.csv")
df.head(10)

,message,category
0,â??@JuanPee24: â??@_MrHandsomeee: â??@tayyoung...,harass
1,@politicalmoxie no dumb ass @Conservativeind c...,harass
2,"@AndrewS82805912 Andrew, Andrew, Andrew.. Tsk ...",harass
3,"Some dumb fuck really had the nerve to type ""f...",harass
4,All I dumb fuck black got all these different ...,harass
5,Bruh they even said he didnâ??t get it threw y...,harass
6,i'm a dumb nigger that loves to leave my twitt...,harass
7,You know there are people out there who like @...,not_harass
8,"@kebesays no, my blog post. :)",not_harass
9,"Kat and Andre celebrating bad cooking, bad att...",not_harass


In [5]:
df_harass = df[df['category']=='harass']
df_not_harass = df[df['category']=='not_harass']

In [6]:
df_not_harass_downsampled = df_not_harass.sample(df_harass.shape[0],replace=True)
df_balanced = pd.concat([df_not_harass_downsampled, df_harass])
df_balanced['category'].value_counts()


not_harass    710
harass        710
Name: category, dtype: int64

In [7]:
df_balanced['harass']=df_balanced['category'].apply(lambda x: 1 if x=='harass' else 0)
df_balanced.sample(2)

,message,category,harass
442,@RealandIllusory = ranting idiot.,not_harass,0
244,@yasmimcaci @Bferrarii PAREM DE FAZER BULLYING...,not_harass,0


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_balanced['message'],df_balanced['harass'], stratify=df_balanced['harass'])
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
model = tf.keras.Model(inputs=[text_input], outputs = [l])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [9]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)
model.fit(X_train, y_train, epochs=12)
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

Epoch 1/12
34/34 [==============================] - 522s 15s/step - loss: 0.6684 - accuracy: 0.5915 - precision: 0.5881 - recall: 0.6135
Epoch 2/12
34/34 [==============================] - 515s 15s/step - loss: 0.5881 - accuracy: 0.7239 - precision: 0.7022 - recall: 0.7786
Epoch 3/12
34/34 [==============================] - 514s 15s/step - loss: 0.5509 - accuracy: 0.7380 - precision: 0.7520 - recall: 0.7111
Epoch 4/12
34/34 [==============================] - 515s 15s/step - loss: 0.5051 - accuracy: 0.8160 - precision: 0.8014 - recall: 0.8405
Epoch 5/12
34/34 [==============================] - 517s 15s/step - loss: 0.4589 - accuracy: 0.8592 - precision: 0.8579 - recall: 0.8612
Epoch 6/12
34/34 [==============================] - 521s 15s/step - loss: 0.4265 - accuracy: 0.8779 - precision: 0.8766 - recall: 0.8799
Epoch 7/12
34/34 [==============================] - 518s 15s/step - loss: 0.4096 - accuracy: 0.8789 - precision: 0.8870 - recall: 0.8687
Epoch 8/12
34/34 [=======================

array([0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1,